In [ ]:
import urllib.request as ur

print("Automatic migration of ProActive catalog ...")

workflowCatalogURL = 'https://try.activeeon.com/workflow-catalog/'

wfURL = 'http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/4/revisions/1?alt=xml'

dlf = open("test.xml", "b+w")
stream = ur.urlopen(wfURL)
dlf.write(stream.read())
dlf.close()

print("Finish Download ...")


In [ ]:
import json
import urllib.request as ur

meta = json.load(ur.urlopen("http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/4"))
#print(meta)
#print(meta.get("_links").get("content").get("href"))
url = meta.get("_links").get("content").get("href")

In [ ]:
import json
import urllib.request as ur

#bucket meta data in the catalog(dict)
metaBucket = json.load(ur.urlopen("http://try.activeeon.com:8080/workflow-catalog/buckets"))
#bucket list(list)
bucketList = metaBucket.get("_embedded").get("bucketMetadataList")
for l in bucketList:
    wfurl = "http://try.activeeon.com:8080/workflow-catalog/buckets/"+str(l.get("id"))+"/workflows"
    #workflow meta data in the buckets (dict)
    metaWorkflow = json.load(ur.urlopen(wfurl))
    #worflow list(list)
    workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
    for w in workflowList:
        print(w.get("name"))


In [ ]:
import json
import urllib.request as ur

bucketURL = "http://try.activeeon.com:8080/workflow-catalog/buckets/1/workflows/"

metaWorkflow = json.load(ur.urlopen(bucketURL))
workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
for w in workflowList:
    wf = bucketURL + str(w.get("id"))
    wfmeta = json.load(ur.urlopen(wf))
    wfURL = wfmeta.get("_links").get("content").get("href")
    file_name = wfmeta.get("name")+".xml"
    u = ur.urlopen(wfURL)
    f = open(file_name, 'wb')
    f.write(u.read())
    f.close()
print("Finish Downloading ...")

In [ ]:
import os
import json
import urllib.request as ur
import zipfile
import shutil

bucket = "http://try.activeeon.com:8080/workflow-catalog/buckets/1"
bucketURL = bucket + "/workflows/"
meta = json.load(ur.urlopen(bucket))
name = meta.get("name")
mypath = "./"+name
if not os.path.isdir(mypath):
   os.makedirs(mypath)

metaWorkflow = json.load(ur.urlopen(bucketURL))
workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
for w in workflowList:
    wf = bucketURL + str(w.get("id"))
    wfmeta = json.load(ur.urlopen(wf))
    wfURL = wfmeta.get("_links").get("content").get("href")
    file_name = wfmeta.get("name")+".xml"
    fullfilename = os.path.join(mypath, file_name)
    u = ur.urlopen(wfURL)
    f = open(fullfilename, 'wb')
    f.write(u.read())
    f.close()
print("Finish Downloading ...")

zipf = zipfile.ZipFile(name+".zip", 'w', zipfile.ZIP_DEFLATED)

for root, dirs, files in os.walk(mypath):
    for file in files:
        zipf.write(os.path.join(root, file))
print("Finish Zipping ...")

if os.path.exists(mypath):
    shutil.rmtree(mypath)
    
print("Original Folder removed ...")

In [2]:
#############################
#ProActive Catalog Migration#
#############################

import os
import sys
import json
import getopt
import shutil
import zipfile
import urllib.request as ur

def downloadWFs(bucket, path):
    path = path + "/catalog"
    bucketURL = bucket + "/workflows/"
    meta = json.load(ur.urlopen(bucket))
    name = meta.get("name")
    mypath = path+"/"+name
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
    metaWorkflow = json.load(ur.urlopen(bucketURL))
    workflowList = metaWorkflow.get("_embedded").get("workflowMetadataList")
    print("Begin Downloading Bucket: " +name+" ...")
    for w in workflowList:
        wf = bucketURL + str(w.get("id"))
        wfmeta = json.load(ur.urlopen(wf))
        wfURL = wfmeta.get("_links").get("content").get("href")
        file_name = wfmeta.get("name")+".xml"
        fullfilename = os.path.join(mypath, file_name)
        u = ur.urlopen(wfURL)
        f = open(fullfilename, 'wb')
        f.write(u.read())
        f.close()
    print("Finish Downloading Bucket: "+name+".")

def zipBucket(bucket, path):
    path = path + "/catalog"
    bucketURL = bucket + "/workflows/"
    meta = json.load(ur.urlopen(bucket))
    name = meta.get("name")
    mypath = path+"/"+name
    zipf = zipfile.ZipFile(mypath+".zip", 'w', zipfile.ZIP_DEFLATED)
    print("Begin Zipping Bucket: " +name+" ...")
    for root, dirs, files in os.walk(mypath):
        for file in files:
            zipf.write(os.path.join(root, file))
    print("Finish Zipping Bucket: "+name+".")
    
    if os.path.exists(mypath):
        shutil.rmtree(mypath)

def migration(catalogURL, path):
    if not os.path.isdir(path):
        os.makedirs(path)    
    bucketsURL = catalogURL+"/buckets"
    metaBucket = json.load(ur.urlopen(bucketsURL))
    bucketList = metaBucket.get("_embedded").get("bucketMetadataList")
    for l in bucketList:
        bucket = bucketsURL+"/"+str(l.get("id"))
        downloadWFs(bucket, path)
        zipBucket(bucket, path)
    print("Migration Finished ...")
    
if __name__ == "__main__":    
    catalogURL = "http://try.activeeon.com:8080/workflow-catalog"
    path = "/Users/sophiesong/Desktop"
    migration(catalogURL, path)

Begin Downloading Bucket: Examples ...
Finish Downloading Bucket: Examples.
Begin Zipping Bucket: Examples ...
Finish Zipping Bucket: Examples.
Begin Downloading Bucket: Cloud-automation ...
Finish Downloading Bucket: Cloud-automation.
Begin Zipping Bucket: Cloud-automation ...
Finish Zipping Bucket: Cloud-automation.
Begin Downloading Bucket: Addons ...
Finish Downloading Bucket: Addons.
Begin Zipping Bucket: Addons ...
Finish Zipping Bucket: Addons.
Migration Finished ...
